In [1]:
import scipy.sparse as spr
import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np


In [104]:
Th = fem.line(0,1,0.005)
print(Th)

< mesh object with 201 nodes, 202 elements of types ( point1, line2 ) >


In [94]:
import pyvista as pv
p = pv.BackgroundPlotter()

grid = Th.to_pv()

p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=16,cmap='jet')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7ffccaac12e8

In [9]:
# The problem to be solved:
# 
# 
Th.elements[1]


{'types': array([1], dtype=int32),
 'tags': [array([3, 4, 5, 6], dtype=uint64)],
 'indices': [array([[0, 2],
         [2, 3],
         [3, 4],
         [4, 1]], dtype=uint64)]}

In [95]:
Th.elements[0]

{'types': array([15], dtype=int32),
 'tags': [array([1, 2], dtype=uint64)],
 'indices': [array([[0],
         [1]], dtype=uint64)]}

In [11]:
elLine = fem.element[1]
elLine.end()
elLine.allocate(2)
elLine.allocate_spatial(1,compute_Jinv = True)


In [18]:
Th.x

matso< shape: (5, 1), 
 - Column 0
(0,0) 0
(1,0) 1
(2,0) 0.25
(3,0) 0.5
(4,0) 0.75
>

In [19]:
elLine.set_coordinates(Th.x,Th.y,Th.z,Th.elements[1]['indices'][0][0])

In [20]:
elLine.compute_jacobian()

In [26]:
oti.gauss_integrate(-oti.dot(oti.transpose(elLine.Nx),elLine.Nx)+oti.dot(oti.transpose(elLine.N),elLine.N),elLine.w_dJ)

matso< shape: (2, 2), 
 - Column 0
(0,0) -3.91667
(1,0) 4.04167
 - Column 1
(0,1) 4.04167
(1,1) -3.91667
>

In [28]:
10*oti.gauss_integrate(oti.transpose(elLine.N),elLine.w_dJ)

matso< shape: (2, 1), 
 - Column 0
(0,0) 1.25
(1,0) 1.25
>

In [30]:

K

<(5, 5) sparse matrix of OTI numbers with 
         0 stored elements in LInked List format>

In [96]:
#*****************************************************************************************************
def solve_1d_problem(Th):
    
    els = Th.elements[1]

    fem.end_elements()
    
    
    K = oti.lil_matso((Th.x.shape[0],Th.x.shape[0]))
    f = oti.zeros((Th.x.shape[0],1))
#     print(f)
#     print(K)
    #integrate 2D a fem array.
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)
            elem.allocate_spatial(1,compute_Jinv = True)

            # Here the temporals shhould be created.
            # - Elemental K
#             Kel = oti.zeros( ( elem.nbasis, elem.nbasis ) )
            # - Elemental f
#             fel = oti.zeros( ( elem.nbasis, 1 ) )
#             tmp1 = oti.zeros(nip=elem.nip)

#             tmp2 = zero()

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
#             Kel.set(0.0); fel.set(0.0)

            elem.set_coordinates(Th.x,Th.y,Th.z,elm_nodes[i,:])
            elem.compute_jacobian()

            Nx = elem.Nx
            N  = elem.N
            
            Kel = oti.gauss_integrate(-oti.dot(oti.transpose(Nx),Nx)+oti.dot(oti.transpose(N),N),elem.w_dJ)
            fel = 10*oti.gauss_integrate(oti.transpose(N),elem.w_dJ)

            assemble_globals(Kel,fel,K,f,elm_nodes[i,:])
            
            # integral = integral + gauss_integrate( dot_product( fh, elem.N), elem.w_dJ)

            # integral = integral + oti.dot_product( fh, oti.gauss_integrate( element.N, element.w_dJ) )

        # end for

    # end for

    fem.end_elements()   
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30

    K[0,0] = TGV
    f[0,0] = 1.0*TGV

    K[1,1] = TGV
    f[1,0] = 4.0*TGV
    
    u = oti.solve(K.tocsr(),f)

    return u,K,f

    #-----------------------------------------------------------------------------------------------------
    
def assemble_globals(Kel,fel,Kgl,fgl,elem):
    
    
    for i in range(elem.size):
        
        ii=int(elem[i])
        
        for j in range(elem.size):

            jj=int(elem[j])
        
            Kgl[ii,jj] = Kgl[ii,jj] + Kel[i,j]
            
        # end for 
        fgl[ii,0] = fgl[ii,0] + fel[i,0]
        
    # end for 
    

In [105]:
u,K,f = solve_1d_problem(Th)
print(u)

matso< shape: (201, 1), 
 - Column 0
(0,0) 1
(1,0) 4
(2,0) 0.993355
(3,0) 0.986935
(4,0) 0.98074
(5,0) 0.974771
(6,0) 0.969028
(7,0) 0.96351
(8,0) 0.958218
(9,0) 0.953152
(10,0) 0.948313
(11,0) 0.943699
(12,0) 0.939313
(13,0) 0.935152
(14,0) 0.931218
(15,0) 0.927511
(16,0) 0.924031
(17,0) 0.920778
(18,0) 0.917751
(19,0) 0.914952
(20,0) 0.91238
(21,0) 0.910035
(22,0) 0.907917
(23,0) 0.906027
(24,0) 0.904364
(25,0) 0.902928
(26,0) 0.90172
(27,0) 0.900739
(28,0) 0.899985
(29,0) 0.89946
(30,0) 0.899161
(31,0) 0.89909
(32,0) 0.899247
(33,0) 0.899631
(34,0) 0.900243
(35,0) 0.901082
(36,0) 0.902149
(37,0) 0.903443
(38,0) 0.904965
(39,0) 0.906714
(40,0) 0.90869
(41,0) 0.910893
(42,0) 0.913324
(43,0) 0.915982
(44,0) 0.918867
(45,0) 0.921979
(46,0) 0.925318
(47,0) 0.928884
(48,0) 0.932677
(49,0) 0.936696
(50,0) 0.940942
(51,0) 0.945414
(52,0) 0.950113
(53,0) 0.955038
(54,0) 0.960189
(55,0) 0.965566
(56,0) 0.971169
(57,0) 0.976998
(58,0) 0.983052
(59,0) 0.989332
(60,0) 0.995837
(61,0) 1.00257
(62

In [110]:
import pyvista as pv
p = pv.BackgroundPlotter()

grid = Th.to_pv(pd=[u.real],pd_names=['u'])

p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
p.add_mesh(grid.warp_by_scalar(), show_edges=True, line_width=2,grid = True,cmap='jet',scalar_bar_args={'interactive':True})
p.show_bounds()
p.show_grid()

In [36]:
K[1,1] = K[1,1] + 3+oti.e(1)
K[1,1]

9.0000 + 3.0000 * e([1])

In [38]:
print(K)

(  1,  1) 9 + 3 * e([1])

